In [ ]:
import os
import time
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
# Импортируйте класс из файла
from preprocess_func_and_classes import BasePeakMarker, read_emg8, split_data

OMG_CHANELS_CNT = 16  # количество каналов OMG-датчиков
OMG_COL_PRFX = 'omg'  # префикс в названиях столбцов датафрейма, соответствующих OMG-датчикам
LABEL_COL = 'id'      # столбец таргета
TS_COL = 'ts'         # столбец метки времени

SYNC_COL = 'sample'   # порядковый номер размеченного жеста - для синхронизации и группировок
TARGET = 'act_label'  # таргет - метка фактически выполняемого жеста

MODEL_PATH = 'model/lstm_model.h5'

# Сформируем список с названиями всех столбцов OMG
OMG_COLS = [OMG_COL_PRFX + str(i) for i in range(OMG_CHANELS_CNT)]

# Функция для предобработки данных
def preprocess_data(data, features):
    marker = BasePeakMarker(bounds_shift=-2)
    marker.fit(data)

    data_marked, bounds = marker._mark(data)

    X_train, X_test, y_train, y_test = split_data(data_marked)

    return X_train, X_test, y_train, y_test

# Функция для создания последовательностей
def create_sequences(data, timesteps=2):
    X = []
    for i in range(len(data) - timesteps + 1):
        X.append(data[i:i + timesteps])
    return np.array(X)

# Функция для предсказания жеста руки
def predict(model, X_test_seq):
    y_pred_encoded = model.predict(X_test_seq)
    y_pred = np.argmax(y_pred_encoded, axis=1)
    return y_pred

# Функция для постобработки и записи результатов в файл
def postprocess_and_write_to_file(predicted_gestures, file_path='logs/results.txt'):
    with open(file_path, 'a') as f:
        for gesture in predicted_gestures:
            # Возвращаем остаток от деления на 10
            processed_gesture = gesture % 10
            f.write(f"Predicted gesture: {processed_gesture}\n")


# Загрузка предобученной модели
model = load_model(MODEL_PATH)

# Основной цикл для обработки данных в реальном времени
def process_realtime_data(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'a') as output_file:
        buffer = []
        timesteps = 3   # Количество строк прошлой информации 
        while True:
            start_time = time.time()
            line = input_file.readline()
            if not line:
                break
            data = list(map(float, line.strip().split(',')))
            buffer.append(data)

            if len(buffer) >= timesteps:
                X_seq = create_sequences(np.array(buffer), timesteps)
                predicted_gestures = predict(model, X_seq)
                postprocess_and_write_to_file(predicted_gestures, output_file_path)
                buffer.pop(0)  # Удаляем первый элемент буфера

            elapsed_time = time.time() - start_time
            sleep_time = max(0.040 - elapsed_time, 0)
            time.sleep(sleep_time)

# Пример использования
input_file_path = 'path/to/your/input/file.csv'
output_file_path = 'logs/results.txt'
process_realtime_data(input_file_path, output_file_path)
